In [ ]:
%pip install yfinance plotly parcllabs==1.10.0

In [2]:
import os
from datetime import datetime

import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from plotly.subplots import make_subplots
from parcllabs.beta.charting.utils import create_labs_logo_dict



client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=1000 # set default limit
)

In [3]:
us = client.search.markets.retrieve(
    query='United States',
    limit=1,
    sort_by='PRICEFEED_MARKET'
)

# get full hist of data
sales_price_feed = client.price_feed.price_feed.retrieve(
    parcl_ids=[us['parcl_id'].values[0]],
    limit=1000,
    auto_paginate=True
)

In [ ]:
symbol = 'HD'
housing_market_name = 'US'
SYMBOL_COLOR_CODE = '#F26722'
SYMBOL_NAME = 'Home Depot'
symbol_df = yf.download(symbol, start="2010-01-01")
# Flatten the multi-index columns from yfinance
symbol_df = symbol_df[["Close"]].droplevel(0, axis=1).rename(columns={"Close": symbol})
symbol_df = symbol_df.reset_index()
symbol_df = symbol_df.rename(columns={'Date': 'date'})

In [5]:
symbol_df = symbol_df.sort_values('date')
sales_price_feed = sales_price_feed.sort_values('date')
merged_df = pd.merge(sales_price_feed, symbol_df, on='date', how='left')

# Forward fill missing values for weekends and holidays
merged_df[symbol] = merged_df[symbol].ffill()
merged_df = merged_df.dropna()
# align series start dates
merged_df = merged_df.loc[merged_df['date'] >= '2010-01-01']

In [ ]:
# Get the last timestamps and calculate percent changes
last_pricefeed_date = merged_df['date'].max().strftime("%Y-%m-%d")
last_vix_datetime = datetime.now().strftime("%Y-%m-%d %H:%M")

# Calculate percent changes
def calc_percent_change(series, days):
    if days == 1:
        # For 1-day change, use the last two values
        return (series.iloc[-1] - series.iloc[-2]) / series.iloc[-2] * 100
    else:
        return (series.iloc[-1] - series.iloc[-days]) / series.iloc[-days] * 100

price_change_1d = calc_percent_change(merged_df['price_feed'], 1)
price_change_7d = calc_percent_change(merged_df['price_feed'], 7)
price_change_30d = calc_percent_change(merged_df['price_feed'], 30)

vix_change_1d = calc_percent_change(merged_df[symbol], 1)
vix_change_7d = calc_percent_change(merged_df[symbol], 7)
vix_change_30d = calc_percent_change(merged_df[symbol], 30)

# Create the dual-axis chart
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add US price per square foot trace
fig.add_trace(
    go.Scatter(
        x=merged_df['date'], 
        y=merged_df['price_feed'], 
        name=f"{housing_market_name} Price per Sq Ft", 
        line=dict(color='#00FFFF', width=3),
        hovertemplate='<b>Date</b>: %{x|%Y-%m-%d}<br><b>Price</b>: $%{y:,.2f}<extra></extra>',
    ),
    secondary_y=False,
)

corr_coef = round(float(merged_df[['price_feed', symbol]].corr()[symbol].values[0]), 4)

# Add VIX trace
fig.add_trace(
    go.Scatter(
        x=merged_df['date'], 
        y=merged_df[symbol], 
        name=symbol, 
        line=dict(color=SYMBOL_COLOR_CODE, width=3),
        hovertemplate="<b>Date</b>: %{x|%Y-%m-%d}<br><b>" + f"{SYMBOL_NAME}" + "</b>: %{y:.2f}<extra></extra>"
    ),
    secondary_y=True,
)

# Set x-axis title and format
fig.update_xaxes(
    title_text="",
    title_font=dict(size=16, color='white'),
    tickfont=dict(size=14, color='white'),
    tickformat='%Y',
    dtick='M24',
    showgrid=True,
    gridcolor='rgba(255,255,255,0.1)'
)

# Set y-axes titles and format
fig.update_yaxes(
    title_text="US Price per Sq Ft ($)", 
    secondary_y=False, 
    title_font=dict(size=16, color='#00FFFF'), 
    tickfont=dict(size=14, color='#00FFFF'),
    tickprefix="$",
    tickformat=",.0f",  # Changed to 2 decimal places
    showgrid=True,
    gridcolor='rgba(0, 255, 255, 0.1)'
)
fig.update_yaxes(
    title_text=SYMBOL_NAME, 
    secondary_y=True, 
    title_font=dict(size=16, color=SYMBOL_COLOR_CODE), 
    tickfont=dict(size=14, color=SYMBOL_COLOR_CODE),
    showgrid=False
)

# Update layout for dark theme and other customizations
fig.update_layout(
    title={
        'text': f"US Housing Prices vs {SYMBOL_NAME} (2010-Present)",
        'y': 0.95,
        'x': 0.05,
        'xanchor': 'left',
        'yanchor': 'top',
        'font': dict(size=28, color='white')
    },
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgb(25,25,25)',
    margin=dict(l=60, r=60, t=120, b=60),
    height=675,
    width=1200,
    hovermode="x unified",
    hoverlabel=dict(
        bgcolor="rgba(0,0,0,0.8)",
        font_size=14,
        font_color="white"
    ),
    showlegend=False,
)

# Add detailed annotation
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.01, y=0.99,
    text=(
        f"<b>US Price per Sq Ft</b> (Last: {last_pricefeed_date})<br>"
        f"1D: <span style='color:#00FFFF'>{price_change_1d:+.4f}%</span> | "  # Changed to 4 decimal places
        f"7D: <span style='color:#00FFFF'>{price_change_7d:+.2f}%</span> | "
        f"30D: <span style='color:#00FFFF'>{price_change_30d:+.2f}%</span><br><br>"
        f"<b>{symbol}</b> (Last: {last_vix_datetime})<br>"
        f"1D: <span style='color:{SYMBOL_COLOR_CODE}'>{vix_change_1d:+.4f}%</span> | "  # Changed to 4 decimal places
        f"7D: <span style='color:{SYMBOL_COLOR_CODE}'>{vix_change_7d:+.2f}%</span> | "
        f"30D: <span style='color:{SYMBOL_COLOR_CODE}'>{vix_change_30d:+.2f}%</span><br><br>"
        f"<b>Correlation Coefficient: <span style='color:{SYMBOL_COLOR_CODE}'>{corr_coef:.4f}</span>"
    ),
    showarrow=False,
    font=dict(size=18, color="white"),
    align="left",
    bgcolor="rgba(0,0,0,0.7)",
    bordercolor="white",
    borderwidth=2,
    borderpad=8,
    yanchor="top"
)

# Add a watermark
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.5, y=0.5,
    text="@ParclLabs",
    showarrow=False,
    font=dict(size=60, color="rgba(255,255,255,0.1)"),
    textangle=-30,
)

fig.add_layout_image(create_labs_logo_dict())


# Show the plot
fig.show()